In [1]:
import ptmpsi
from ptmpsi.alphafold import prediction
from ptmpsi.protein import Protein
from ptmpsi.nwchem import get_qm_data
import os

tutorial_directory = os.path.join(os.getcwd())
machine='frontier'
account='bip258'
long_partition='extended'
gromacs_options=ptmpsi.slurm.frontier.partitions[long_partition].options["gromacs"]

# Structure Prediction with AlphaFold

## Pass a FASTA sequence to the `prediction` function

In [ ]:
submission_directory = os.path.join(tutorial_directory, 'alphafold_submission')
os.makedirs(submission_directory, exist_ok=True)
os.chdir(submission_directory)
prediction("GGGGGGGGGGG", multimer=False, machine=machine, account=account, partition=long_partition, ngpus=8, time=8)
os.chdir(tutorial_directory)

Three files will be generated in the ./alphafold_submission directory: `alphafold.sbatch`, `run_singularity.py` and `temp1.fasta`.
Paths to the required databases will default to their location in the specified machine.

The `prediction` function also accepts a list of sequences for the sequential prediction for each one of the sequences.

If **multimer** is instead desired, one must pass `multimer=True` to the `prediction` and a path to 
a file containing two or more sequences with the following format:

```
>sequence_1
GGGGGGGG

>sequence_2
AAAAAAAA
```

An example of submitting the alphafold job is shown below.

In [ ]:
os.chdir(submission_directory)
os.system('sbatch alphafold.sbatch')
os.chdir(tutorial_directory)

# Loading a predicted 3D structure

The `Protein` class handles all protein/polymer operations. We will use this class to load or build the 3D structure 
of a polymer. 

## Fetch a structure from the Protein Data Bank
Only works when internet access is available. Create a `Protein` object using the `pdbid` option to pass a Protein Data Bank ID code.

In [ ]:
fetched_pdb = Protein(pdbid="6tht")

## Fetch a structure from the AlphaFold Database
Only works when internet access is available. Create a `Protein` object using the `uniprotid` option to pass a Uniprot ID code.

In [ ]:
uniprot_pdb = Protein(uniprotid="G9BY57")

## Read a structure from a local PDB file
Create a `Protein` object using the `filename` option to pass the path to a local PDB file

In [ ]:
local_pdb = Protein(filename="G9BY57.pdb")

## Build a chain from scratch
Create an empty `Protein` object and use the `append` and `prepend` functions to add residues to a given polymer *chain*. The `phi`/`psi` amide torsions can be customized while using the `prepend`/`append` commands, respectively. The default values will produce torsion angles compatible with the alpha-helix conformation. 

In [9]:
chain = Protein()

# Build a trimer of epsilon-amino caproic acid
chain.prepend(chain="A", residue="EAC")
chain.append(chain="A", residue="EAC", psi=120.0)
chain.append(chain="A", residue="EAC", psi=120.0)

# Cap the trimer to have neutral ends
chain.prepend(chain="A", residue="ACE", phi=-120.0)
chain.append(chain="A", residue="NME", psi=120.0)

## Generate a Nylon-4 or Nylon-6 surface
Use the `polymers.nylon4.gen_surface` or the `polymers.nylon6.gen_surface` function to create a PDB file containing a finite cluster approximation of the corresponding surface by specifying the number of unit cells to include along the `a`, `b`, and `c` directions. Two PDB files will be generated, one where all strands share the same *MODEL* parameter and another where each strand is its own *MODEL*. There is no difference among the generated surfaces, but some visualization software packages might get confused with many strands in one single *MODEL*.

Three different Nylon-6 crystallographic parameters can be selected with the `model` keyword: "Malta1979", "Holmes", or "Arimoto".

There is only one Nylon-4 `model` available.

In [ ]:
nylon_pdb_dirs = os.path.join(tutorial_directory, 'nylon_pdbs')
os.makedirs(nylon_pdb_dirs, exist_ok=True)
os.chdir(nylon_pdb_dirs)
ptmpsi.polymers.nylon6.gen_surface(rangea=3, rangeb=3, rangec=3, model="Malta1979")
ptmpsi.polymers.nylon4.gen_surface(rangea=3, rangeb=3, rangec=3)
os.chdir(tutorial_directory)

# Introducing point mutations

The `mutate` function of the `Protein` class can be used to introduce point mutations in a protein. For example, the active form of the ICCG variant of cutinase can be obtained by the following commands.

In [ ]:
cutinase = Protein(pdbid="6tht")
cutinase.mutate("A:ALA130","SER")

The residue to be mutated can be specified with the nomenclature "\[chain\]:\[name\]\[number\]", where \[chain\] is the one-letter chain identifier, \[name\] is the three-letter aminoacid name, and \[number\] is the residue number. The \[name\] can be omitted.

# Obtaining AMBER99 parameters for a residue

The `get_qm_data` function will produce a series of `NWChem` input files that will generate `RESP` charges and bonded force constants according to the `AMBER99` forcefield recipe. The current output is obtained in `STDOUT` and the force constants are given in a format that can be copied directly into a `GROMACS` forcefield definition.

By default, `get_qm_data` will generate two conformations for the new residue capped by an `ACE` and `NME` groups. The `RESP` fitting for both conformers will be performed simultaneously and only one set of charges will be computed. Bond and angle force constants parameters will be generated **separately** and the user must average these values in order to obtain a single constant for each bond or angle.

When needed, torsion potentials are fitted for both conformations at the same time.

In [2]:
eac = Protein()
eac.prepend(chain="A", residue="EAC")
qm_path = os.path.join(os.getcwd(), "get_qm_data_nwchem_submission")
get_qm_data(eac, path=qm_path, machine=machine, account=account, partition=long_partition, time=8)


Some NWChem input options can be changed via keyword arguments. These options, along with their default values, are:

```
mult    = kwargs.get("mult",1)
charge  = kwargs.get("charge",0)
memory  = kwargs.get("memory",2000)
aobasis = kwargs.get("aobasis","def2-tzvp")
tdbasis = kwargs.get("tdbasis","def2-svp")
cdbasis = kwargs.get("cdbasis","def2-universal-jfit")
xcfun   = kwargs.get("xcfun","r2scan")
grid    = kwargs.get("grid","lebedev 120 14")
tdgrid  = kwargs.get("tdgrid","lebedev 100 14")
nscf    = kwargs.get("nscf",100)
nopt    = kwargs.get("nopt",60)
disp    = kwargs.get("disp","disp vdw 4")
delta   = kwargs.get("delta",0.0189)
lshift  = kwargs.get("lshift",0.1)
```

An example of running the command is shown below.

In [6]:
os.chdir(qm_path)
os.system('sbatch get_qm_data.sbatch')
os.chdir(tutorial_directory)

# Post-translational modifications

Post-translational modifications can be introduced by using the `modify` function of the `Protein` class. The following is a list of available CYS post-translational modifications
 - nitrosylation
 - sulfenylation
 - sulfynilation
 - sulfonylation
 - sulfhydration
 - glutathionylation
 - cysteinylation
 - methylation
 - carbamoylation
 - cyanylation

The following post-translational modifications are also available for a subset of aminoacids
 - acetylation
     - LYS, LYN
 - methylation:
     - GLU, GLH
     - LYS, LYN
     - ARG
     - HIS, HIP, HID, HIE
 - dimethylation:
     - LYS, LYN, ARG
 - trimethylation:
     - LYS, LYN
 - symmetric dimethylation:
     - ARG
 - asymmetric dimethylation:
     - ARG        
 - phosphorylation:
     - SER
     - THR
     - TYR
     - ARG
     - HIS, HIP, HID, HIE
     - LYS, LYN
     - ASP, ASH
     - CYS, CYM

In [4]:
snc = Protein()
snc.prepend(chain="A", residue="CYS")
snc.modify("A:CYS1", "nitrosylation")

nitrosylation SNC

	 Found rotamer with no clashes!


# Docking

PTM-Psi relies on an existing installation of AutoDock Vina.

In [4]:
docking_dir = os.path.join(tutorial_directory, 'docking_output')
os.makedirs(docking_dir, exist_ok=True)
os.chdir(docking_dir)
cutinase = Protein(pdbid="6tht")
cutinase.mutate("A:ALA130","SER")
cutinase.write_pdb("cutinase.pdb")

nylon6 = Protein()
nylon6.prepend("A", "EAC")
nylon6.prepend("A", "ACE")
nylon6.append("A", "EAC")
nylon6.append("A", "NME")
nylon6.write_xyz("nylon6.xyz")
cutinase.dock(ligand="nylon6.xyz", receptor="cutinase.pdb", boxcenter="A:SER130", boxsize=20.0)
os.chdir(tutorial_directory)

		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.58
		 Selecting location 'A' with occupancy   0.57
		 Selecting location 'A' with occupancy   0.57
		 Selecting location 'A' with occupancy   0.57
		 Selecting location 'B' with occupancy   0.63
		 Selecting location 'B' with occupancy   0.63
		 Selecting location 'B' with occupancy   0.63
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'B' with occupancy   0.60
		 Selecting location 'B' with occupancy   0.60
		 Selecting location 'B' with occupancy   0.60
		 Selecting location 'A' with occupancy   0.60
		 Selecting location 'A' with occupancy

!!! Warning !!! Distance between atoms 10 and 9 (0.996594 A) is suspicious.
!!! Warning !!! Distance between atoms 13 and 12 (0.998549 A) is suspicious.
!!! Warning !!! Distance between atoms 14 and 12 (0.993680 A) is suspicious.
!!! Warning !!! Distance between atoms 16 and 15 (0.996845 A) is suspicious.
!!! Warning !!! Distance between atoms 20 and 18 (0.993680 A) is suspicious.
!!! Warning !!! Distance between atoms 22 and 21 (0.997196 A) is suspicious.
!!! Warning !!! Distance between atoms 29 and 28 (0.996594 A) is suspicious.
!!! Warning !!! Distance between atoms 32 and 31 (0.998549 A) is suspicious.
!!! Warning !!! Distance between atoms 33 and 31 (0.993680 A) is suspicious.
!!! Warning !!! Distance between atoms 35 and 34 (0.996845 A) is suspicious.
!!! Warning !!! Distance between atoms 39 and 37 (0.993680 A) is suspicious.
!!! Warning !!! Distance between atoms 41 and 40 (0.997196 A) is suspicious.
/ccs/proj/bip258/reeceneff/ptmpsi-env/lib/python3.11/site-packages/meeko/mols

templates=<meeko.linked_rdkit_chorizo.ResidueChemTemplates object at 0x7f9e01010650>


[18:27:56] product atom-mapping number 11 not found in reactants.
[18:27:56] product atom-mapping number 12 not found in reactants.
[18:27:56] product atom-mapping number 11 not found in reactants.
[18:27:56] product atom-mapping number 12 not found in reactants.
[18:27:56] product atom-mapping number 13 not found in reactants.
[18:27:56] product atom-mapping number 11 not found in reactants.
[18:27:56] product atom-mapping number 12 not found in reactants.



Files written:
cutinase.pdbqt <-- static (i.e., rigid) receptor input file
AutoDock Vina v1.2.5
#################################################################
# If you used AutoDock Vina in your work, please cite:          #
#                                                               #
# J. Eberhardt, D. Santos-Martins, A. F. Tillack, and S. Forli  #
# AutoDock Vina 1.2.0: New Docking Methods, Expanded Force      #
# Field, and Python Bindings, J. Chem. Inf. Model. (2021)       #
# DOI 10.1021/acs.jcim.1c00203                                  #
#                                                               #
# O. Trott, A. J. Olson,                                        #
# AutoDock Vina: improving the speed and accuracy of docking    #
# with a new scoring function, efficient optimization and       #
# multithreading, J. Comp. Chem. (2010)                         #
# DOI 10.1002/jcc.21334                                         #
#                                            

# NWChem QM/MM input generation
Note: this run takes over 18 hours.

In [5]:
os.chdir(docking_dir)
cutinase.protonate()
cutinase.write_pdb("cutinase.pdb")

ptmpsi.docking.write_pdb(cutinase.docking, filename="test2.pdb", pose=0, template=nylon6)
test2_path = os.path.join(os.getcwd(), "test2.pdb")
qmmm_submission = os.path.join(tutorial_directory, 'qmmm_submission')
ptmpsi.nwchem.qmmm_optimize(filename=test2_path, qmres=[130,207,259,260,261,262], counter=True, totcharge=cutinase.charge, xcfun="pbe96", grid="medium", aobasis="def2-svp",  path=qmmm_submission, account=account, machine=machine, partition=long_partition, time=24, nnodes=2)
os.chdir(tutorial_directory)

Below is an example for submitting `qmmm_optimization.sbatch`

In [6]:
os.chdir(qmmm_submission)
os.system('sbatch qmmm_optimization.sbatch')
os.chdir(tutorial_directory)

Submitted batch job 2619273


# Generate post-translational modification combinations

- Load the GAP2/CP12/PRK complex

In [7]:
protein = Protein(pdbid="6gve")

- Find all cysteine residues

In [8]:
protein.findresidue("CYS")

D:CYS19
D:CYS67
D:CYS77
D:CYS154
D:CYS158
A:CYS19
A:CYS67
A:CYS77
A:CYS154
A:CYS158
E:CYS19
E:CYS67
E:CYS77
E:CYS154
E:CYS158
F:CYS19
F:CYS67
F:CYS77
F:CYS154
F:CYS158
I:CYS19
I:CYS67
I:CYS77
I:CYS154
I:CYS158
K:CYS19
K:CYS67
K:CYS77
K:CYS154
K:CYS158
N:CYS19
N:CYS67
N:CYS77
N:CYS154
N:CYS158
O:CYS19
O:CYS67
O:CYS77
O:CYS154
O:CYS158


- Generate all posible single combinations (ntuple=1) for nitrosylation and sulfhydration

In [9]:
combinations1 = protein.get_ptm_combinations([("CYS", "nitrosylation"),("CYS","sulfhydration")], ntuple=1)

	Number of 1-tuple combinations: 80
Total number of combinations: 80


- List all generated combinations

In [10]:
print("Residue         PTM")
for ptm in combinations1[0]:
    print(f"{ptm[0][0]:10s}   {ptm[0][1]:20s}")

Residue         PTM
D:CYS19      nitrosylation       
D:CYS67      nitrosylation       
D:CYS77      nitrosylation       
D:CYS154     nitrosylation       
D:CYS158     nitrosylation       
A:CYS19      nitrosylation       
A:CYS67      nitrosylation       
A:CYS77      nitrosylation       
A:CYS154     nitrosylation       
A:CYS158     nitrosylation       
E:CYS19      nitrosylation       
E:CYS67      nitrosylation       
E:CYS77      nitrosylation       
E:CYS154     nitrosylation       
E:CYS158     nitrosylation       
F:CYS19      nitrosylation       
F:CYS67      nitrosylation       
F:CYS77      nitrosylation       
F:CYS154     nitrosylation       
F:CYS158     nitrosylation       
I:CYS19      nitrosylation       
I:CYS67      nitrosylation       
I:CYS77      nitrosylation       
I:CYS154     nitrosylation       
I:CYS158     nitrosylation       
K:CYS19      nitrosylation       
K:CYS67      nitrosylation       
K:CYS77      nitrosylation       
K:CYS154     nitrosylation  

- Generate all possible single and double combinations excluding modification of all CYS19 and CYS77 of GAP2

In [11]:
exclude = [ f"{chain}:{res}" for chain in ["A", "D", "E", "F", "I", "K", "N", "O"] for res in ["CYS19", "CYS77"] ]
combinations2 = protein.get_ptm_combinations([("CYS", "nitrosylation"),("CYS","sulfhydration")], exclude=exclude, ntuple=2)

	Number of 1-tuple combinations: 48
	Number of 2-tuple combinations: 1104
Total number of combinations: 1152


- Find all cysteine residues involved in disulfide bonds (identifed as CYX)

In [12]:
protein.findresidue("CYX")

P:CYX19
P:CYX29
P:CYX61
P:CYX70
J:CYX13
J:CYX35
J:CYX224
J:CYX230
B:CYX13
B:CYX35
B:CYX224
B:CYX230
G:CYX13
G:CYX35
G:CYX224
G:CYX230
L:CYX13
L:CYX35
L:CYX224
L:CYX230
C:CYX19
C:CYX29
C:CYX61
C:CYX70
H:CYX19
H:CYX29
H:CYX61
H:CYX70
M:CYX19
M:CYX29
M:CYX61
M:CYX70


- Generate a combination list that reduces both disulfide bonds of CP12 chain M

In [13]:
combinations3 = protein.get_ptm_combinations([("M:CYX", "reduction")], ntuple=4)

	Number of 1-tuple combinations: 4
	Number of 2-tuple combinations: 6
	Number of 3-tuple combinations: 4
	Number of 4-tuple combinations: 1
Total number of combinations: 15


In [14]:
print("1-tuple")
print("     Residue         PTM")
for i,ptm in enumerate(combinations3[0]):
    for residue in ptm:
        print(f"{i+1:3d}  {residue[0]:10s}   {residue[1]:20s}")

print()
print("2-tuple")
print("     Residue1       PTM1     Residue2       PTM2")
for i,ptm in enumerate(combinations3[1]):
    print(f"{i+1:3d}   ", end="")
    for residue in ptm:
        print(f"{residue[0]}    {residue[1]}    ", end="")
    print()

print()
print("3-tuple")
print("     Residue1       PTM1     Residue2       PTM2     Residue3      PTM3")
for i,ptm in enumerate(combinations3[2]):
    print(f"{i+1:3d}   ", end="")
    for residue in ptm:
        print(f"{residue[0]}    {residue[1]}    ", end="")
    print()

print()
print("4-tuple")
print("     Residue1       PTM1     Residue2       PTM2     Residue3      PTM3      Residue4      PTM4")
for i,ptm in enumerate(combinations3[3]):
    print(f"{i+1:3d}   ", end="")
    for residue in ptm:
        print(f"{residue[0]}    {residue[1]}    ", end="")
    print()


1-tuple
     Residue         PTM
  1  M:CYX19      reduction           
  2  M:CYX29      reduction           
  3  M:CYX61      reduction           
  4  M:CYX70      reduction           

2-tuple
     Residue1       PTM1     Residue2       PTM2
  1   M:CYX19    reduction    M:CYX29    reduction    
  2   M:CYX19    reduction    M:CYX61    reduction    
  3   M:CYX19    reduction    M:CYX70    reduction    
  4   M:CYX29    reduction    M:CYX61    reduction    
  5   M:CYX29    reduction    M:CYX70    reduction    
  6   M:CYX61    reduction    M:CYX70    reduction    

3-tuple
     Residue1       PTM1     Residue2       PTM2     Residue3      PTM3
  1   M:CYX19    reduction    M:CYX29    reduction    M:CYX61    reduction    
  2   M:CYX19    reduction    M:CYX29    reduction    M:CYX70    reduction    
  3   M:CYX19    reduction    M:CYX61    reduction    M:CYX70    reduction    
  4   M:CYX29    reduction    M:CYX61    reduction    M:CYX70    reduction    

4-tuple
     Residue1    

Note that some double (2-tuple) combinations and the quadruple (4-tuple) combination are the only ones that make sense for this case as CYX19 is bonded to CYX29 and CYX61 is bonded to CYX70.

- Generate the run files for the quadruple combination

The following command will generate a folder inside gromacs_sims with a unique identifier (UID) with all the necessary files to submit a full set of GROMACS MD simulations including energy minimization, equilibration, and a 100 ns production run. Note that there is a modified **specbond.dat** file that will prevent GROMACS from forming a disulfide bond between spatially close **CYS** residues, but it will still form disulfide bonds between two nearby **CYX** residues.

In [15]:
gromacs_path = os.path.join(os.getcwd(), "gromacs_sims")
uid = protein.gen_ptm_files([combinations3[3]], lenmd=1, lennvt=100, lennpt=100, path=gromacs_path, machine=machine, account=account, **gromacs_options, partition="extended", time=8)
print(f'\nExecute the following command to run the simulation: `cd {os.path.join(gromacs_path, uid)} && ./submit.sh`')

	 Reading local PDB file
reduction CYS

	 Found rotamer with no clashes!
reduction CYS

	 Current rotamer has 4 possible clashes
	 Current rotamer has 4 possible clashes
	 Current rotamer has 3 possible clashes
	 Current rotamer has 3 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 3 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 1 possible clashes
	 Current rotamer has 1 possible clashes

	 Found rotamer with no clashes!
reduction CYS

	 Found rotamer with no clashes!
reduction CYS

	 Current rotamer has 2 possible clashes
	 Current rotamer has 1 possible clashes
	 Current rotamer has 2 possible clashes
	 Current rotamer has 2 possible cla

The following command will execute the submission script to automatically sbatch
the necessary files.

In [17]:
os.chdir(os.path.join(gromacs_path, uid))
os.system("chmod +x submit.sh")
os.system("./submit.sh")
os.chdir(tutorial_directory)

Submitted batch job 2619471
Submitted batch job 2619473
Submitted batch job 2619475
Submitted batch job 2619477
Submitted batch job 2619479
Submitted batch job 2619482
Submitted batch job 2619484
Submitted batch job 2619485
Submitted batch job 2619487
Submitted batch job 2619489
Submitted batch job 2619491
Submitted batch job 2619493
Submitted batch job 2619495
